In [1]:
import datetime
datetime.datetime.utcnow()
import logging
import numpy as np
logging.getLogger('hax.minitrees').setLevel(logging.INFO)
import hax
import root_pandas
import lax
import pandas as pd
from s2_top_bottom_new import S2TopBottom
import os
from hax import cuts

DEBUG:InterpolatingMap] Loading JSON map /home/amb243/s2_xy_XENON1T_24Feb2017.json
DEBUG:InterpolatingMap] Map name: XENON1T S2(x, y) correction map
DEBUG:InterpolatingMap] Map description:
    XENON1T S2 xy correction map
    Version 1.0
    Created 24th February 2017 by Adam Brown
    
    Extracted from November 2016 Kr83m calibration (summed 41.6 keV peak)
    Using 84 datasets totalling 2 days 22:05:00 between 2016-11-14 10:39:38 and 2016-11-18 06:38:52
    Made using data processed with pax v6.4.2
    
    See note: xenon:xenon1t:adam:fv-study:s2-correction-xy-kr83m.html
    
DEBUG:InterpolatingMap] Map names found: ['map', 'map_bottom', 'map_top']


In [2]:
#pax_version = '6.6.2'
pax_version = '6.4.2'
sciencerun = 0
directory = '/project/lgrandi/tunnell/data/sciencerun%d/pax_%s/lax_%s' % (sciencerun,
                                                                         pax_version,
                                                                         lax.__version__)

print(directory)
if not os.path.exists(directory):
    os.makedirs(directory)
    
excluded = [] #if pax_version == '6.4.2' else ["161104_1351", '161109_0502',
              #                                '161117_1132', '161210_1003', 
              #                                '161214_1036', '170109_1323', 
              #                                '170110_0843', '170116_0537']

/project/lgrandi/tunnell/data/sciencerun0/pax_6.4.2/lax_0.11.1


In [3]:
hax.init(experiment='XENON1T')
hax.runs.update_datasets({"tags.name" : '_sciencerun%d' % sciencerun,
                          "name" : {"$nin" : excluded},
                          "data": {"$elemMatch": {"status": 'transferred',
                                                  'type': 'processed',
                                                  'host' : 'midway-login1',
                                                   'pax_version': 'v%s' % pax_version}}
                         })

datasets = hax.runs.datasets 

In [4]:
from lax.lichens import sciencerun0
print("Ready. Steady. Go!")
bad = []

Ready. Steady. Go!


In [5]:
lookup_table = {row.number : row.source__type for i, row in datasets.T.iteritems()}
lookup_table2 = {row.number : '_sciencerun0,' in row.tags for i, row in datasets.T.iteritems()}    
lookup_table3 = {row.number : 'closed' in row.tags for i, row in datasets.T.iteritems()}    
lookup_table4 = {row.number : row['name'] for i, row in datasets.T.iteritems()} 

In [6]:
from lax.lichens import sciencerun0
cut = {'Rn220' : sciencerun0.LowEnergyRn220(),
       'AmBe' : sciencerun0.LowEnergyAmBe(),
       'none' : sciencerun0.LowEnergyBackground(),
      'Kr83m' : sciencerun0.LowEnergyBackground()}
       

In [7]:
preselections = {#'1t_fiducial' : sciencerun0.LowEnergyBackground().lichen_list[0].string,
                'cs1_lt_200' : 'cs1<%d' % sciencerun0.LowEnergyBackground().lichen_list[1].allowed_range[1],
}

#preselections['1t_fiducial_cs1_lt_200'] = '(%s) & (%s)' % (preselections['1t_fiducial'],
#                                                           preselections['cs1_lt_200'])

preselections['lowenergy'] = preselections['cs1_lt_200']
preselections['lowenergy_roi'] = '(cs1 > 3) & (cs1 < 70)'

In [8]:
minitrees = ['Fundamentals', 'Basics', 'TotalProperties',
             'Proximity', 'Extended', 'TailCut']
if pax_version == '6.4.2': 
    minitrees.append(S2TopBottom)

for preselection_name, preselection in preselections.items():
    print(preselection_name, preselection)
    hax.init(experiment='XENON1T',
             main_data_paths=['/project2/lgrandi/xenon1t/processed/pax_v%s' % pax_version,
                              '/project/lgrandi/xenon1t/processed/pax_v%s' % pax_version],
             minitree_paths=['/project/lgrandi/xenon1t/minitrees/pax_v%s' % pax_version,
                             '/project2/lgrandi/xenon1t/minitrees/pax_v%s' % pax_version,
                            '/project/lgrandi/brown/s2tb_minitrees',
                            # '.'
                            ],
             #make_minitrees=True,#False if pax_version == '6.4.2' else True,
             pax_version_policy=pax_version)

    for source_type in datasets.source__type.unique(): 
        if source_type != 'AmBe':
            continue
        print(source_type)
        df = hax.minitrees.load(datasets[datasets.source__type == source_type].name,
                                treemakers=minitrees,
                                num_workers=20,
                                preselection=[preselection]
                                )

        df.loc[:, 'source'] = [lookup_table[x] for x in list(df.loc[:, 'run_number'])]
        df.loc[:, 'name'] = [lookup_table4[x] for x in list(df.loc[:, 'run_number'])]
        df.loc[:, 'pax_version'] = pax_version
        df.loc[:, 'lax_version'] = lax.__version__

        for source_cut in cut.values():
            df = source_cut.process(df)
        df = cut[source_type].process(df)

        base = '%s/data_%s_%s' % (directory,
                          source_type,
                          preselection_name)
        
        if 'lowenergy' in preselection_name:
            for each in cut[source_type].lichen_list:
                cut_name = 'Cut' + each.__class__.__name__
                df = cuts.selection(df, df[cut_name],
                                    desc='%s v%d' % (cut_name, each.version))
            
            df.to_csv(base + '.csv')

        store = pd.HDFStore(base + '.h5',
                            complib='blosc',
                            complevel=9)
        store.put('data', df)

        # Store the cuts history for the data
        store.get_storer('data').attrs.cut_history = cuts._get_history(df)
        store.close()

        df.to_root(base + '.root', 'tree')

lowenergy cs1<200
AmBe


/project/lgrandi/anaconda3/envs/pax_head/lib/python3.4/site-packages/pandas/computation/align.py:98: RuntimeWarning: divide by zero encountered in log10
  ordm = np.log10(abs(reindexer_size - term_axis_size))


cs1<200 selection: 12776709 rows removed (1.34% passed)
CutAmBeFiducial v1 selection: 127982 rows removed (26.13% passed)
CutS1LowEnergyRange v0 selection: 0 rows removed (100.00% passed)
CutS2Threshold v1 selection: 16018 rows removed (64.62% passed)
CutInteractionPeaksBiggest v0 selection: 625 rows removed (97.86% passed)
CutS2AreaFractionTop v2 selection: 871 rows removed (96.96% passed)
CutS2SingleScatterSimple v0 selection: 20832 rows removed (24.96% passed)
CutDAQVeto v1 selection: 640 rows removed (90.76% passed)
CutS1SingleScatter v1 selection: 37 rows removed (99.41% passed)
CutS1AreaFractionTop v1 selection: 56 rows removed (99.10% passed)
CutS2PatternLikelihood v0 selection: 349 rows removed (94.37% passed)
CutS2Tails v0 selection: 293 rows removed (94.99% passed)
CutS1PatternLikelihood v0 selection: 303 rows removed (94.54% passed)
CutS2Width v2 selection: 309 rows removed (94.12% passed)
CutS1MaxPMT v0 selection: 49 rows removed (99.01% passed)
CutSingleElectronS2s v3 sele

/project/lgrandi/anaconda3/envs/pax_head/lib/python3.4/site-packages/root_numpy/_tree.py:449: UserWarning: converter for dtype('O') is not implemented (skipping)
  _librootnumpy.array2root(arr, filename, treename, mode)


cs1_lt_200 cs1<200
AmBe
cs1<200 selection: 12776709 rows removed (1.34% passed)
lowenergy_roi (cs1 > 3) & (cs1 < 70)
AmBe
(cs1 > 3) & (cs1 < 70) selection: 12828299 rows removed (0.94% passed)
CutAmBeFiducial v1 selection: 84690 rows removed (30.39% passed)
CutS1LowEnergyRange v0 selection: 0 rows removed (100.00% passed)
CutS2Threshold v1 selection: 15449 rows removed (58.22% passed)
CutInteractionPeaksBiggest v0 selection: 579 rows removed (97.31% passed)
CutS2AreaFractionTop v2 selection: 645 rows removed (96.92% passed)
CutS2SingleScatterSimple v0 selection: 15586 rows removed (23.24% passed)
CutDAQVeto v1 selection: 405 rows removed (91.42% passed)
CutS1SingleScatter v1 selection: 25 rows removed (99.42% passed)
CutS1AreaFractionTop v1 selection: 28 rows removed (99.35% passed)
CutS2PatternLikelihood v0 selection: 251 rows removed (94.11% passed)
CutS2Tails v0 selection: 190 rows removed (95.26% passed)
CutS1PatternLikelihood v0 selection: 146 rows removed (96.18% passed)
CutS2Wid

In [9]:
print('done')

done
